In [200]:
import torch as t
import math as m
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
import matplotlib.gridspec as gridspec

from matplotlib.font_manager import FontProperties

In [201]:
# get data
words = open("names.txt", 'r').read().splitlines()

In [202]:
# lookup tables for s --> i || i --> s
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s,i in stoi.items()}
vocab_size = len(itos)

In [203]:
# build the dataset
block_size = 3

def build_dataset(data,
                  stoi: dict,
                  itos: dict,
                  block_size: int):
    """
    creates X tensor which contains input tensors, and Y tensor, which contains the associated output
    :param block_size: number of necessary characters to predict the next tone
    :param itos: lookup table from string to int
    :param stoi: lookup table from int to string
    :param data: list of names, nouns, etc.
    :return: tuple of X and Y
    """
    X, Y = [], []
    for w in data:

        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix]    # crop and append

    X = t.tensor(X)
    Y = t.tensor(Y)
    # print(X.shape, Y.shape)

    return X, Y


random.seed(1789)
random.shuffle(words)

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

# create splits
Xtr,  Ytr  = build_dataset(words[:n1], stoi=stoi, itos=itos, block_size=block_size)      # 80%
Xdev, Ydev = build_dataset(words[n1:n2], stoi=stoi, itos=itos, block_size=block_size)    # 10%
Xte,  Yte  = build_dataset(words[n2:], stoi=stoi, itos=itos, block_size=block_size)      # 10%

In [204]:
def cmp(s, dt, te):
    """
    used to compare manually calculated gradients to PyTorch's gradients
    """
    ex = t.all(dt == te.grad).item()
    app = t.allclose(dt, te.grad)
    maxdiff = (dt - te.grad).abs().max().item()
    print(f"{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}")

In [205]:
# MLP revisited
n_emb = 10       # dimensionality of the character embedding vectors
n_hidden = 64    # number of neurons in the hidden layer of the MLP

g  = t.Generator().manual_seed(1789)
C  = t.randn((vocab_size, n_emb),            generator=g)

# Layer 1
w1 = t.randn((n_emb * block_size, n_hidden), generator=g) * (5/3)/((n_emb * block_size)**0.5)
b1 = t.randn(n_hidden,                       generator=g) * 0.1

# Layer 2
w2 = t.randn((n_hidden, vocab_size),         generator=g) * 0.1
b2 = t.randn(vocab_size,                     generator=g) * 0.1

# BatchNorm parameters
bngain = t.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = t.randn((1, n_hidden)) * 0.1

parameters = [C, w1, b1, w2, b2, bngain, bnbias]
print(f"number of parameters in total: {sum(p.nelement() for p in parameters)}")

for p in parameters:
    p.requires_grad = True

number of parameters in total: 4137


In [240]:
batch_size = 32
n = batch_size
ix = t.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [241]:
# forward pass, explicited into smaller steps that are possible to backward one at a time
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

# Linear Layer 1
hprebn = embcat @ w1 + b1

# BatchNorm Layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-Linearity
h = t.tanh(hpreact)

# Linear Layer 2
logits = h @ w2 + b2

# Cross Entropy Loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
    p.grad = None

for e in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
    e.retain_grad()
loss.backward()
loss

tensor(3.4270, grad_fn=<NegBackward0>)

In [244]:
# manual backward pass

# loss
dlogprobs = t.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n
cmp('logprobs', dlogprobs, logprobs)

# logprobs
dprobs = (1.0/probs) * dlogprobs
cmp('probs', dprobs, probs)

# probs
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
dcounts = counts_sum_inv * dprobs
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# counts_sum_inv
dcounts_sum = -counts_sum**-2 * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

# counts_sum
dcounts += t.ones_like(counts) * dcounts_sum
cmp('counts', dcounts, counts)

# counts
dnorm_logits = norm_logits.exp() * dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

# norm_logits
dlogits = dnorm_logits
dlogit_maxes = -dnorm_logits.sum(1, keepdim=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)

# logit_maxes
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp('logits', dlogits, logits)

# Linear Layer 2
# logits
dh = dlogits @ w2.T
dw2 = h.T @ dlogits
db2 = dlogits.sum(0)
cmp('w2', dw2, w2)
cmp('b2', db2, b2)
cmp('h', dh, h)

# Non-Linearity
dhpreact = (1.0 - h**2) * dh
cmp('hpreact', dhpreact, hpreact)

# hpreact
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('bngain', dbngain, bngain)
cmp('bnraw', dbnraw, bnraw)
cmp('bnbias', dbnbias, bnbias)

# BatchNorm Layer
# bnraw
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

# bnvar_inv
dbnvar = -0.5 * (bnvar + 1e-5)**-1.5 * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

# bnvar
dbndiff2 = 1.0/(n-1) * t.ones_like(bndiff2) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)

# bndiff2
dbndiff += (2 * bndiff) * dbndiff2
cmp('bndiff', dbndiff, bndiff)

# bndiff
dhprebn = dbndiff.clone()
dbnmeani = -dbndiff.sum(0, keepdim=True)
cmp('bnmeani', dbnmeani, bnmeani)

# bnmeani
dhprebn += 1.0/n * t.ones_like(hprebn) * dbnmeani
cmp('hprebn', dhprebn, hprebn)

# Linear Layer 1
# hprebn
dembcat = dhprebn @ w1.T
dw1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
cmp('w1', dw1, w1)
cmp('b1', db1, b1)
cmp('embcat', dembcat, embcat)

# embcat
demb = dembcat.view(emb.shape)
cmp('emb', demb, emb)

# emb
dC = t.zeros_like(C)
dC = dC.index_add_(0, Xb.view(-1),  demb.view(-1, 10))
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
w2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff:

In [245]:
# backward pass

# Cross Entropy Loss
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n
cmp('logits', dlogits, logits)

# BatchNorm Layer



logits          | exact: False | approximate: True  | maxdiff: 6.05359673500061e-09
